### Necessary imports

In [3]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
#!pip install datasets

### Dependencies

In [4]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [5]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer

In [1]:
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00


In [2]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00


### Load quantized Zephyr 7B

In [7]:
#################################################################
# Tokenizer
#################################################################

#model_name='mistralai/Mistral-7B-Instruct-v0.1'
model_name='TheBloke/zephyr-7B-beta-GPTQ'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
bnb_config = GPTQConfig(bits=4,
                        disable_exllama=True,
                        device_map="auto",
                        use_cache=False,
                        lora_r=16,
                        lora_alpha=16,
                        tokenizer=tokenizer
                                )
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "TheBloke/zephyr-7B-beta-GPTQ"
#model_name_or_path = "TheBloke/notus-7B-v1-GPTQ"
print(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", trust_remote_code=False)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
#model = AutoModelForCausalLM.from_pretrained(
#    model_name,
#    quantization_config=bnb_config,
#)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

### Count number of trainable parameters

In [8]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 262410240
percentage of trainable model parameters: 100.00%


### Build Zephyr text generation pipeline

In [9]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=50,
)

In [10]:
Zephyr_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

### Load and chunk documents. Load chunked documents into FAISS index

In [ ]:
#!playwright install
#!playwright install-deps

153.1 Mb [] 0% 0.0s153.1 Mb [] 0% 31.2s153.1 Mb [] 0% 18.0s153.1 Mb [] 0% 15.0s153.1 Mb [] 0% 17.9s153.1 Mb [] 0% 13.8s153.1 Mb [] 1% 9.8s153.1 Mb [] 1% 8.7s153.1 Mb [] 1% 7.2s153.1 Mb [] 2% 7.1s153.1 Mb [] 2% 6.4s153.1 Mb [] 3% 6.1s153.1 Mb [] 3% 5.9s153.1 Mb [] 4% 5.3s153.1 Mb [] 5% 4.8s153.1 Mb [] 5% 4.4s153.1 Mb [] 6% 4.1s153.1 Mb [] 7% 4.0s153.1 Mb [] 7% 3.9s153.1 Mb [] 8% 3.9s153.1 Mb [] 8% 4.0s153.1 Mb [] 8% 3.9s153.1 Mb [] 9% 3.7s153.1 Mb [] 10% 3.5s153.1 Mb [] 11% 3.3s153.1 Mb [] 12% 3.3s153.1 Mb [] 12% 3.2s153.1 Mb [] 13% 3.1s153.1 Mb [] 14% 3.1s153.1 Mb [] 15% 3.0s153.1 Mb [] 15% 3.1s153.1 Mb [] 15% 3.2s153.1 Mb [] 16% 3.2s153.1 Mb [] 17% 3.1s153.1 Mb [] 18% 3.0s153.1 Mb [] 19% 3.0s153.1 Mb [] 19% 2.9s153.1 Mb [] 20% 2.8s153.1 Mb [] 21% 2.8s153.1 Mb [] 21% 2.7s153.1 Mb [] 22% 2.7s153.1 Mb [] 23% 2.7s153.1 Mb [] 23% 2.6s153.1 Mb [] 24% 2.6s153.1 Mb [] 25% 2.6s153.1 Mb [] 25% 2.7s153.1 Mb [] 26% 2.6s153.1 Mb [] 26% 2.7s153.1 Mb [] 27% 2.6s153.1 Mb [] 28% 2.5s153.1 Mb [] 29% 2.

In [26]:
from langchain.document_loaders import AmazonTextractPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [30]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


#loader_story_part1 = CSVLoader("/content/sample_data/simple_story_question_answers_for_finetuning.csv")
#loader_story_part2 = CSVLoader("/content/sample_data/simple_extended_story_question_answers_for_rag.csv")



In [56]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader


import os , time
pwd=os.getcwd()
st=time.time()

loader=DirectoryLoader(path=pwd,glob="**/simple*.csv",use_multithreading=True, loader_cls=TextLoader)
documents=loader.load()
endt=time.time()
et=endt-st
print(et)
print(len(docs))

0.0029518604278564453
2


In [40]:
#documents = loader_all.load()

In [57]:
# Chunk text to keep it meaningful size
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

all_chunks = []

document = loader.load()
chunks = splitter.split_documents(document)
all_chunks += chunks

In [58]:
embedding_model_id = "BAAI/bge-small-en-v1.5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
)

In [59]:
# Embed chunks
db = FAISS.from_documents(all_chunks, embeddings)


In [60]:
retriever = db.as_retriever()

### Create PromptTemplate and LLMChain

In [18]:
##### [INST] Instruction: Answer the question based on your fantasy football knowledge. Here is context to help:

prompt_template = """
### [INST] Instruction: Answer the question based on your  knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=Zephyr_llm, prompt=prompt)

### Build RAG Chain

In [19]:
# Ask Question 1
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("Where did Vendhan and Maria organize the procession inspired by Panguni Uthiram?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [20]:
# Retrives the relavant data from the questionaire dataset loaded into the Vector DB FAISS
# It takes our input question (Query), converts into a sentence embedding vector and searches
#for the closet possible vectors in the DB which are similar to the question asked.
result['context']

[Document(page_content='\ufeffQuestion: Where did Vendhan and Maria organize the procession inspired by Panguni Uthiram?\nAnswer: Lisbon', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 13}),
 Document(page_content='\ufeffQuestion: Where did Vendhan and Maria organize the procession inspired by Panguni Uthiram?\nAnswer: Lisbon', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 34}),
 Document(page_content='\ufeffQuestion: Where did Vendhan and Maria organize the procession inspired by Panguni Uthiram?\nAnswer: Lisbon', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 55}),
 Document(page_content='\ufeffQuestion: Where did Vendhan and Maria organize the procession inspired by Panguni Uthiram?\nAnswer: Lisbon', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 76})]

In [21]:
# Answer to the Queston 1
print(result['text'])


<|assistant|>
According to the given context, Vendhan and Maria organized the procession inspired by Panguni Uthiram in Lisbon. This information is provided as the answer to the same question in four


In [23]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What was the universal language that Vendhan and Maria discovered?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [24]:
result['context']

[Document(page_content='\ufeffQuestion: What was the universal language that Vendhan and Maria discovered?\nAnswer: Love and compassion', metadata={'source': '/content/sample_data/simple_extended_story_question_answers.csv', 'row': 16}),
 Document(page_content='\ufeffQuestion: What was the universal language that Vendhan and Maria discovered?\nAnswer: Love and compassion', metadata={'source': '/content/sample_data/simple_extended_story_question_answers.csv', 'row': 37}),
 Document(page_content='\ufeffQuestion: What was the universal language that Vendhan and Maria discovered?\nAnswer: Love and compassion', metadata={'source': '/content/sample_data/simple_extended_story_question_answers.csv', 'row': 58}),
 Document(page_content='\ufeffQuestion: What was the universal language that Vendhan and Maria discovered?\nAnswer: Love and compassion', metadata={'source': '/content/sample_data/simple_extended_story_question_answers.csv', 'row': 79})]

In [25]:
print(result['text'])


<|assistant|>
The universal language that Vendhan and Maria discovered is love and compassion, as stated in the answers provided for this question in the given context. This language transcends cultural and linguistic barriers and connects people


In [22]:
#Question 3
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("Who is Aadhan?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
result['context']

[Document(page_content='\ufeffQuestion: What was Vendhan captivated by during Panguni Uthiram?\nAnswer: Fervent devotion of the devotees', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 11}),
 Document(page_content='\ufeffQuestion: What was Vendhan captivated by during Panguni Uthiram?\nAnswer: Fervent devotion of the devotees', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 32}),
 Document(page_content='\ufeffQuestion: What was Vendhan captivated by during Panguni Uthiram?\nAnswer: Fervent devotion of the devotees', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 53}),
 Document(page_content='\ufeffQuestion: What was Vendhan captivated by during Panguni Uthiram?\nAnswer: Fervent devotion of the devotees', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv', 'row': 74})]

In [24]:
print(result['text'])


<|assistant|>
The provided context does not offer any information about who Aadhan is. It appears to be a separate question from the ones regarding Vendhan's experience during Panguni Uthiram. Without


In [61]:
#Question 4
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("Who is Aadhan?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [62]:
result['context']

[Document(page_content='who is Aadhan?,The Emperror of Kandigai\nWho is Aadhan?,Aadhan is a noble king.', metadata={'source': '/content/sample_data/simple_extended_story_question_answers_for_rag.csv'}),
 Document(page_content='Who faced assassination attempts?,Aadhan', metadata={'source': '/content/sample_data/simple_story_question_answers_for_finetuning.csv'}),
 Document(page_content='Who encountered challenges in solidifying his rule?,Aadhan', metadata={'source': '/content/sample_data/simple_story_question_answers_for_finetuning.csv'}),
 Document(page_content='Who faced challenges in reclaiming his destiny?,Aadhan', metadata={'source': '/content/sample_data/simple_story_question_answers_for_finetuning.csv'})]

In [63]:
print(result['text'])


### ANSWER:
Aadhan is a noble king, as mentioned in the given context.
